# 1. Data Preprocessing

## 1.1. Data Loading

## To fully utilize the data features, we concatenate the text with its corresponding prompt. Try to extract features from the concatenated data. ##

In [1]:
import pandas as pd

# Reading data files
essays_df = pd.read_csv('llm-detect-ai-generated-text/train_essays.csv')
prompts_df = pd.read_csv('llm-detect-ai-generated-text/train_prompts.csv')

# Link text with corresponding prompts
merged_df = essays_df.merge(prompts_df, on='prompt_id', how='left')
merged_df['text'] = merged_df['prompt_name'] + '\n\n' + merged_df['instructions'] + '\n\n' + merged_df['text']
merged_df.drop(merged_df.columns[[4, 5, 6]],axis=1, inplace=True)

merged_df.to_csv('llm-detect-ai-generated-text/train_merged.csv', index=False)

In [2]:
merged_df.head(3)

,id,prompt_id,text,generated
0,0059830c,0,Car-free cities\n\nWrite an explanatory essay ...,0
1,005db917,0,Car-free cities\n\nWrite an explanatory essay ...,0
2,008f63e3,0,Car-free cities\n\nWrite an explanatory essay ...,0


In [3]:
print(f"Numbers of essays: {merged_df.id.unique().shape[0]}")

Numbers of essays: 1378


In [4]:
txtdata = []
classes = []
for row in merged_df.itertuples():
    txtdata.append(row[3])
    classes.append(row[4])

## 1.2. Data Division

### Since there is no verifiable test set in the dataset, in order to evaluate the model, we re-divide the training set into a training set and a test set, where 20% of the data is used for the test evaluation. ###

In [5]:
from sklearn import *

# Split the data set using the train_test_split method， 20% for test sets
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(txtdata, classes, test_size=0.2, random_state=42)


In [6]:
print("data type:", type(train_data))
print("label type:", type(train_labels))
print("train size:",len(train_data))
print("test size:",len(test_data))

data type: <class 'list'>
label type: <class 'list'>
train size: 1102
test size: 276


# 2. Feature Extraction

## 2.1. TF-IDF

## 2.2. Sentence Vector

In [7]:
# Load Vector Model
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")

In [8]:
# Get sentence vector
train_feature = model.encode(train_data, show_progress_bar=True)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [10]:
print(f'Shape of the vectorised abstract: {train_feature[0].shape}')
print(f'Shape of the embeddings: {train_feature.shape}')

Shape of the vectorised abstract: (768,)
Shape of the embeddings: (1102, 768)


# 3. Model Training

## 3.1. Logistic Regression

In [ ]:
# learn logistic regression classifier using CV
from numpy import *
logreg = linear_model.LogisticRegressionCV(Cs=logspace(-4,4,20), cv=5, n_jobs=-1)
logreg.fit(train_feature, train_labels)

print("w=", logreg.coef_)
print("b=", logreg.intercept_)

## 3.2. XGBoost

# 4. Evaluation

## 4.1. Accuracy

In [ ]:
# calculate accuracy
acc = metrics.accuracy_score(test_labels, predY)
print("test accuracy=", acc)

## 4.2. ROC Curve

# 5. Optimization

# 6. Analysis and ComparisonS